In [183]:
# Import the required modules
import pandas as pd
import numpy as np

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# Deep Learning!
import tensorflow as tf

# Metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, RocCurveDisplay

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier


In [184]:
# Import pandas and read the charity_data.csv from the provided cloud URL.
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
print(df.shape)
df.head()

(34299, 12)


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [185]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = df.drop(columns=["EIN", "NAME"])

# Display the first few rows to confirm changes
print(df.shape)
df.head()

(34299, 10)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [187]:
# Determine the number of unique values in each column.
unique_values = df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [188]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# Check the value counts of APPLICATION_TYPE
application_type_counts = df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [189]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff_value = 500

# Get the count of unique values in the 'APPLICATION_TYPE' column
value_counts = df['APPLICATION_TYPE'].value_counts()

# Create a list of application types that appear less than the cutoff value
application_types_to_replace = value_counts[value_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
df['APPLICATION_TYPE'].value_counts()

# Check to make sure the replacement was successful
print(df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [190]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
# Check the value counts of CLASSIFICATION
classification_counts = df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [191]:
# Determine a cutoff value
cutoff = 500  

# Create a list of classifications to be replaced
classification_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace classifications in the dataframe
for cls in classification_to_replace:
   df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls, "Other")

# Verify the changes
print(df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64


In [192]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
# Check the value counts of CLASSIFICATION
classification_counts = df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64


In [193]:
# Determine a cutoff value
cutoff = 500  

# Create a list of classifications to be replaced
classification_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace classifications in the dataframe
for cls in classification_to_replace:
   df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls, "Other")

# Verify the changes
print(df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64


In [194]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# Check value counts of CLASSIFICATION and filter those appearing more than once
temp = df.CLASSIFICATION.value_counts()
temp.loc[temp > 1]
# Display classifications with counts greater than 1
classification_counts_filtered = classification_counts[classification_counts > 1]
print(classification_counts_filtered)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64


In [195]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Determine a cutoff value for classification
cutoff = 500  

# Create a list of classifications to be replaced
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace classifications in the dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls, "Other")

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")

df_encoded = pd.get_dummies(df, dtype=int)
df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [196]:
# Check to make sure replacement was successful
df.IS_SUCCESSFUL.value_counts()

IS_SUCCESSFUL
1    18261
0    16038
Name: count, dtype: int64

In [197]:
# Convert categorical variables to numeric using one-hot encoding
application_df = pd.get_dummies(df, dtype=int)

In [198]:
# Split our preprocessed data into our features and target arrays
X = df_encoded.drop(['IS_SUCCESSFUL'], axis=1)
y = df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 30, stratify=y)

print(X.shape)
print(X_train.shape)
print(X_test.shape)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

(34299, 44)
(29154, 44)
(5145, 44)


In [199]:
def do_classification_nn(nn_model, X_train, X_test, y_train, y_test):
    # Predict probabilities
    train_proba = nn_model.predict(X_train)
    test_proba = nn_model.predict(X_test)
    
    # Convert probabilities to binary predictions
    train_preds = (train_proba >= 0.5).astype(int)
    test_preds = (test_proba >= 0.5).astype(int)
    
    # Ensure labels are NumPy arrays
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    # Compute metrics for training set
    train_cf = confusion_matrix(y_train, train_preds)
    train_cr = classification_report(y_train, train_preds, digits=4)
    train_auc = roc_auc_score(y_train, train_proba)
    
    train_results = f"""TRAIN METRICS
    Confusion Matrix:
    {train_cf}
    
    AUC: {train_auc:.4f}
    
    Classification Report:
    {train_cr}
    """
    
    # Compute metrics for test set
    test_cf = confusion_matrix(y_test, test_preds)
    test_cr = classification_report(y_test, test_preds, digits=4)
    test_auc = roc_auc_score(y_test, test_proba)
    
    test_results = f"""TEST METRICS
    Confusion Matrix:
    {test_cf}
    
    AUC: {test_auc:.4f}
    
    Classification Report:
    {test_cr}
    """
    
    print(train_results)
    print(test_results)
    
    return {
        "train": {
            "confusion_matrix": train_cf,
            "auc": train_auc,
            "classification_report": train_cr
        },
        "test": {
            "confusion_matrix": test_cf,
            "auc": test_auc,
            "classification_report": test_cr
        }
    }

In [200]:
 # Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=len(X.columns)))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

/opt/anaconda3/envs/dev/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 5)              │           225 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 231 (924.00 B)

 Trainable params: 231 (924.00 B)

 Non-trainable params: 0 (0.00 B)

In [201]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [202]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=15)

Epoch 1/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 511us/step - accuracy: 0.5104 - loss: 24452.2363
Epoch 2/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step - accuracy: 0.5261 - loss: 2157.7068
Epoch 3/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step - accuracy: 0.5148 - loss: 4257.1465
Epoch 4/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - accuracy: 0.5246 - loss: 10567.5322
Epoch 5/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step - accuracy: 0.5170 - loss: 10874.2549
Epoch 6/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step - accuracy: 0.5434 - loss: 7601.3784
Epoch 7/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.5176 - loss: 15863.0410
Epoch 8/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - accuracy: 0.5522 - loss: 3698.4197
Epoch 9/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.5765 - loss: 2832.9692
Epoch 10/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step - accuracy: 0.5561 - loss: 25429.8145
Epoch 11/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - accuracy: 0.5472 - loss: 26

In [203]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

161/161 - 0s - 1ms/step - accuracy: 0.7071 - loss: 3.0433
Loss: 3.0432748794555664, Accuracy: 0.7070942521095276


In [204]:
# Export our model to HDF5 file
nn_model.save('AAlphabetSoupCharity_Optimization.h5')